In [ ]:
# | default_exp classes.DomoDataflow_History

In [ ]:
# | exporti

from __future__ import annotations

from dataclasses import dataclass, field
from typing import List
import datetime as dt

import httpx

from nbdev.showdoc import patch_to

import domolibrary.utils.convert as ct
import domolibrary.utils.chunk_execution as ce

import domolibrary.client.DomoAuth as dmda
import domolibrary.routes.dataflow as dataflow_routes

In [ ]:
# | export
from domolibrary.classes.DomoDataflow_Action import DomoDataflow_ActionResult

In [ ]:
# | hide
from nbdev.showdoc import show_doc

# Dataflow_History_Execution

class that models a dataflow execution event.
can include list of `DomoDataflow_ActionResult` which are tile executions


In [ ]:
# | export
@dataclass
class DomoDataflow_History_Execution:
    auth: dmda.DomoAuth = field(repr=False)
    id: str
    dataflow_id: str
    dataflow_execution_id: str
    dataflow_version: str

    begin_time: dt.datetime
    end_time: dt.datetime
    last_updated: dt.datetime

    is_failed: bool
    state: str
    activation_type: str
    data_processor: str
    telemetry: dict
    execution_stats: dict

    action_results: List[DomoDataflow_ActionResult] = None

    @classmethod
    def _from_json(cls, de_obj, auth: dmda.DomoAuth):

        action_results = None
        if de_obj.get("actionResults"):
            action_results = [
                DomoDataflow_ActionResult._from_json(action_obj)
                for action_obj in de_obj.get("actionResults")
            ]

        return cls(
            auth=auth,
            id=de_obj["id"],
            dataflow_id=de_obj["onboardFlowId"],
            dataflow_execution_id=de_obj["dapDataFlowExecutionId"],
            dataflow_version=de_obj.get("dataFlowVersion"),
            begin_time=ct.convert_epoch_millisecond_to_datetime(
                de_obj.get("beginTime")
            ),
            end_time=ct.convert_epoch_millisecond_to_datetime(de_obj.get("endTime")),
            last_updated=ct.convert_epoch_millisecond_to_datetime(
                de_obj["lastUpdated"]
            ),
            is_failed=de_obj.get("failed"),
            state=de_obj["state"],
            activation_type=de_obj["activationType"],
            data_processor=de_obj["dataProcessor"],
            telemetry=de_obj.get("telemetry"),
            execution_stats={
                "total_bytes_written": de_obj.get("totalBytesWritten", 0),
                "total_rows_read": de_obj.get("totalRowsRead", 0),
                "total_bytes_read": de_obj.get("totalBytesRead", 0),
                "mean_download_rate_kbps": de_obj.get("meanDownloadRateKbps", 0),
                "total_rows_written": de_obj.get("totalRowsWritten", 0),
            },
            action_results=action_results,
        )

In [ ]:
# | exporti


@patch_to(DomoDataflow_History_Execution, cls_method=True)
async def get_by_id(
    cls,
    auth: dmda.DomoAuth,
    dataflow_id: int,
    execution_id: int,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    """retrieves details about a dataflow execution including actions"""

    res = await dataflow_routes.get_dataflow_execution_by_id(
        auth=auth,
        dataflow_id=dataflow_id,
        execution_id=execution_id,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=cls.__name__,
        session=session,
    )

    if return_raw:
        return res

    return cls._from_json(auth=auth, de_obj=res.response)

In [ ]:
show_doc(DomoDataflow_History_Execution.get_by_id)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/classes/DomoDataflow_History.py#L86){target="_blank" style="float:right; font-size:smaller"}

### DomoDataflow_History_Execution.get_by_id

>      DomoDataflow_History_Execution.get_by_id
>                                                (auth:domolibrary.client.DomoAu
>                                                th.DomoAuth, dataflow_id:int,
>                                                execution_id:int,
>                                                debug_api:bool=False,
>                                                debug_num_stacks_to_drop=1,
>                                                session:httpx.AsyncClient=None,
>                                                return_raw:bool=False)

retrieves details about a dataflow execution including actions

#### sample implementation of Dataflow_History_Execution get_by_id


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

from pprint import pprint

pprint(
    await DomoDataflow_History_Execution.get_by_id(
        dataflow_id=70, execution_id=433238, auth=token_auth, return_raw=False
    )
)

DomoDataflow_History_Execution(id=433238,
                               dataflow_id=70,
                               dataflow_execution_id='88e96b86-0dcd-4a84-a946-bcb22224ea2f',
                               dataflow_version=262,
                               begin_time=datetime.datetime(2023, 12, 5, 6, 16, 31),
                               end_time=datetime.datetime(2023, 12, 5, 6, 17, 4),
                               last_updated=datetime.datetime(2023, 12, 5, 6, 17, 4),
                               is_failed=False,
                               state='SUCCESS',
                               activation_type='MANUAL',
                               data_processor='MYSQL',
                               telemetry=None,
                               execution_stats={'mean_download_rate_kbps': 56.014071115596686,
                                                'total_bytes_read': 291266,
                                                'total_bytes_written': 97986,
        

In [ ]:
# | exporti


@patch_to(DomoDataflow_History_Execution)
async def get_actions(
    self: DomoDataflow_History_Execution,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    """retrieves details execution action results"""

    res = await dataflow_routes.get_dataflow_execution_by_id(
        auth=self.auth,
        dataflow_id=self.dataflow_id,
        execution_id=self.id,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=self.__class__.__name__,
        session=session,
    )

    if return_raw:
        return res

    action_results = res.response.get("actionResults")
    if action_results:
        action_results = [
            DomoDataflow_ActionResult._from_json(action_obj)
            for action_obj in action_results
        ]

    self.action_results = action_results
    return self.action_results

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

from pprint import pprint

domo_dataflow_execution = await DomoDataflow_History_Execution.get_by_id(
    dataflow_id=70, execution_id=433238, auth=token_auth, return_raw=False
)
await domo_dataflow_execution.get_actions()

[DomoDataflow_ActionResult(id='8edc3d78-3586-47a8-9775-c7b146a50d19', type='PublishToVault', depends_on=None, name=None, datasource_id=None, sql=None, is_success=True, rows_processed=1816, begin_time=datetime.datetime(2023, 12, 5, 6, 17, 1, 487000), end_time=datetime.datetime(2023, 12, 5, 6, 17, 2, 754000), duration_in_sec=1.267),
 DomoDataflow_ActionResult(id='c176aa53-122a-4a2e-8b6f-41ca4a72d67c', type='GenerateTableAction', depends_on=None, name=None, datasource_id=None, sql=None, is_success=True, rows_processed=909, begin_time=datetime.datetime(2023, 12, 5, 6, 17, 0, 983000), end_time=datetime.datetime(2023, 12, 5, 6, 17, 1, 100000), duration_in_sec=0.117),
 DomoDataflow_ActionResult(id='241025d7-3cca-4369-b7c0-b3264277c0e1', type='DataHubManifestLoaderAction', depends_on=None, name=None, datasource_id=None, sql=None, is_success=True, rows_processed=909, begin_time=datetime.datetime(2023, 12, 5, 6, 16, 55, 894000), end_time=datetime.datetime(2023, 12, 5, 6, 17, 0, 972000), duration

In [ ]:
# | export
@dataclass
class DomoDataflow_History:
    auth: dmda.DomoAuth = field(repr=False)
    dataflow_id: int = field(repr=False)

    dataflow: None = field(repr=False, default=None)

    execution_history: List[DomoDataflow_History_Execution] = None

In [ ]:
# | exporti
@patch_to(DomoDataflow_History)
async def get_execution_history(
    self: DomoDataflow_History,
    auth: dmda.DomoAuth = None,
    maximum=10,  # maximum number of execution histories to retrieve
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    return_raw: bool = False,
):
    """retrieves metadata about execution history.
    includes details like execution status.
    """

    auth = auth or self.auth or self.dataflow.auth

    res = await dataflow_routes.get_dataflow_execution_history(
        auth=auth,
        dataflow_id=self.dataflow_id,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=self.__class__.__name__,
        maximum=maximum,
    )

    if return_raw:
        return res

    execution_history = [
        DomoDataflow_History_Execution._from_json(df_obj, auth)
        for df_obj in res.response
    ]

    await ce.gather_with_concurrency(
        *[domo_execution.get_actions() for domo_execution in execution_history], n=20
    )

    self.execution_history = execution_history

    return self.execution_history

In [ ]:
show_doc(DomoDataflow_History.get_execution_history)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/classes/DomoDataflow_History.py#L158){target="_blank" style="float:right; font-size:smaller"}

### DomoDataflow_History.get_execution_history

>      DomoDataflow_History.get_execution_history
>                                                  (auth:domolibrary.client.Domo
>                                                  Auth.DomoAuth=None,
>                                                  maximum=10,
>                                                  debug_api:bool=False,
>                                                  debug_num_stacks_to_drop=2,
>                                                  return_raw:bool=False)

retrieves metadata about execution history.
includes details like execution status.

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_dataflow_history = DomoDataflow_History(dataflow_id=185, auth=token_auth)

await domo_dataflow_history.get_execution_history(debug_api=False, return_raw=False)

[DomoDataflow_History_Execution(id=327615, dataflow_id=185, dataflow_execution_id='98fb0606-fa1b-48ea-bf96-0f3a052b7e99', dataflow_version=741, begin_time=datetime.datetime(2023, 6, 15, 21, 17, 58), end_time=datetime.datetime(2023, 6, 15, 21, 18, 9), last_updated=datetime.datetime(2023, 6, 15, 21, 18, 9), is_failed=False, state='SUCCESS', activation_type='MANUAL', data_processor='MAGIC', telemetry=[{'name': 'activeOptimizations', 'value': 'c'}], execution_stats={'total_bytes_written': 2299757, 'total_rows_read': 5988, 'total_bytes_read': 624402, 'mean_download_rate_kbps': 239.78276764648052, 'total_rows_written': 5988}, action_results=[DomoDataflow_ActionResult(id='b9bb0f01-2517-4595-a077-c9778e5ffc48', type='DataHubManifestLoaderAction', depends_on=None, name=None, datasource_id=None, sql=None, is_success=True, rows_processed=5988, begin_time=datetime.datetime(2023, 6, 15, 21, 18, 5, 716000), end_time=datetime.datetime(2023, 6, 15, 21, 18, 7, 8000), duration_in_sec=1.292), DomoDataflo

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()